In [1]:
import tools
import param
import cartesian
import racks
import calibration
import samples
import json
import importlib
import random as rnd
import time
import low_level_comm as llc

In [2]:
llc.listSerialPorts()

['COM3', 'COM6', 'COM10', 'COM14']

In [3]:
ar = cartesian.arnie(cartesian_port="COM10", docker_port="COM6")

In [4]:
ar.home()

In [5]:
stp = tools.stationary_touch_probe(ar)

ERROR:root:matchPortsWithDevices(): could not execute finiding device in list.
ERROR:root:matchPortsWithDevices(): Provided patterns_list: ['stationary touch probe']
ERROR:root:None


# Initializing 96 well racks for the 1st time

In [6]:
# Have to provide slot numbers
rack96well = racks.rack(rack_name='96well_rack', rack_type='96wells_rack', x_slot=2, y_slot=3)
# Magnetic rack has the same geometry as normal rack, so the same rack type is used
magrack96well = racks.rack(rack_name='96well_rack_magnetic', rack_type='96wells_rack', x_slot=3, y_slot=3)

In [12]:
rack96well.rack_data

{'name': '96well_rack',
 'n_x': 2,
 'n_y': 3,
 'type': '96wells_rack',
 'position': [392.75, 395.85, 507.65],
 'pos_stalagmyte': [94.0, 65.85, 490.55]}

In [9]:
# No info regarding its coordinates present, we have to calibrate it for the first time.
tp = tools.mobile_touch_probe.getTool(ar)

In [10]:
x, y, z = calibration.calibrateStationaryProbe(tp, stp)

In [10]:
xr, yr, zr = calibration.calibrateRack(tp, rack96well)
xr, yr, zr = calibration.calibrateRack(tp, magrack96well)

NameError: name 'tp' is not defined

In [13]:
tp.returnTool()

In [14]:
ar.home()

In [16]:
# Now robot knows the position of the center.
# It will be used to calculate individual wells positions.
rack96well.rack_data

{'name': '96well_rack',
 'n_x': 2,
 'n_y': 3,
 'type': '96wells_rack',
 'position': [392.75, 395.85, 507.65],
 'pos_stalagmyte': [94.0, 65.85, 490.55]}

# Introducing PCR tube

In [28]:
# I have to create a new json file with all the parameters
# This is the relation of liquid volume and the distance from the liquid leve to the top of the tube
# The tube has no markings, so I have to fill the tube with liquid and manually measure its level
vol_vs_z = {
    0: 20.0,
    50: 13.77,
    80: 11.24,
    100: 10.30,
    150: 8.26,
    200: 5.88,
    250: 3.50,
    300: 1.40,
}

In [29]:
# Now this is the value on how much the tube extends above the rack
sample_top_dz = {
    "96wells_rack": 7.5,
}

In [30]:
# Info on how to grab the tube with a gripper
# Those parameters need to be found experimentally as well
gripper_param = {
    "open_to_approach_uncapped": 8,
    "grip_uncapped": 4,
}

In [31]:
param = {
    'volume_vs_z': vol_vs_z,
    'sample_top_dz': sample_top_dz,
    'gripper_param': gripper_param,
        }

In [32]:
with open('configs/300uL_PCR_tube_params.json', 'w') as fp:
    json.dump(param, fp)

## Getting gripper

In [7]:
#gr = tools.mobile_gripper.getTool(robot=ar)
gr = tools.mobile_gripper(robot=ar)

In [8]:
calibration.calibrateToolCustomPoints(tool=gr, stationary_probe=stp)

(95.1, 66.575, 515.4)

In [9]:
ar.move(z=300)

In [28]:
# Manually acquiring x and y positions
gr.getToPosition(rack=rack96well, column=5, row=3)

In [38]:
# Finding how wide to open claws
gr.operateGripper(0)

In [35]:
ar.move(z=520)

In [65]:
# Initializing sample
s1 = samples.sample(sample_name='s1', sample_type='PCR_tube_300uL')

In [66]:
s1.sample_data

{'sample_name': 's1', 'sample_type': 'PCR_tube_300uL'}

In [67]:
s1.place(rack96well, 5, 3)

In [72]:
gr.grabSample(s1)

In [73]:
gr.placeSample(rack96well, 5, 3)

In [33]:
# Playing with several samples:
sample_number = 6
sample_list = []
for i in range(sample_number):
    # Initializing samples
    s = samples.sample(sample_name='s'+str(sample_number), sample_type='PCR_tube_300uL')
    # Placing them into the rack
    # row 7 (starting from 0), column variable
    s.place(rack96well, i, 7)
    sample_list.append(s)

In [34]:
sample_list

In [78]:
for s in sample_list:
    col_number = s.sample_data['x_well']
    gr.moveSample(s, magrack96well, col_number, 7)

In [79]:
for s in sample_list:
    col_number = s.sample_data['x_well']
    gr.moveSample(s, rack96well, col_number, 7)

In [17]:
ar.home()

# Testing sample pushing procedure

In [12]:
gr.pushSample(sample_list[1], dz=3)

In [35]:
for s in sample_list:
    col_number = s.sample_data['x_well']
    gr.moveSample(s, magrack96well, col_number, 7, push_sample=True, push_dz=3, push_open_diam=0)

In [ ]:
for s in sample_list:
    col_number = s.sample_data['x_well']
    gr.moveSample(s, rack96well, col_number, 7, push_sample=True, push_dz=3, push_open_diam=0)

In [25]:
gr.moveSample(sample_list[2], magrack96well, 3, 7, push_sample=True, push_dz=3, push_open_diam=0)
gr.moveSample(sample_list[2], rack96well, 3, 7, push_sample=True, push_dz=3, push_open_diam=0)